# Style-Bert-VITS2ライブラリの使用例

`pip install style-bert-vits2`を使った、jupyter notebookでの使用例です。Google colab等でも動きます。

In [2]:
# PyTorch環境の構築（ない場合）
# 参照: https://pytorch.org/get-started/locally/
# CUDAを使用するPyTorchをインストール（GPU計算を高速化）
# GPU環境がない場合はCPU版が自動でインストールされる

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
# style-bert-vits2のインストール

!pip install style-bert-vits2

In [4]:
# BERTモデルをロード（ローカルに手動でダウンロードする必要はありません）
# テキストを理解するためのBERTモデルと、テキストをトークンに分割するトークナイザーをロード

from style_bert_vits2.nlp import bert_models
from style_bert_vits2.constants import Languages

# 日本語用のBERTモデルとトークナイザーをロード
# DeBERTaは日本語の文字レベルで訓練された高性能なモデル
bert_models.load_model(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")
bert_models.load_tokenizer(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")

# 英語用（コメントアウト中）
# bert_models.load_model(Languages.EN, "microsoft/deberta-v3-large")
# bert_models.load_tokenizer(Languages.EN, "microsoft/deberta-v3-large")

# 中国語用（コメントアウト中）
# bert_models.load_model(Languages.ZH, "hfl/chinese-roberta-wwm-ext-large")
# bert_models.load_tokenizer(Languages.ZH, "hfl/chinese-roberta-wwm-ext-large")

/Users/Aporo/Study/python/text-to-speach/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1119 00:36:57.757000 3159 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


11-19 00:36:59 |  INFO  | bert_models.py:92 | Loaded the Languages.JP BERT model from ku-nlp/deberta-v2-large-japanese-char-wwm
11-19 00:36:59 |  INFO  | bert_models.py:154 | Loaded the Languages.JP BERT tokenizer from ku-nlp/deberta-v2-large-japanese-char-wwm


BertJapaneseTokenizer(name_or_path='ku-nlp/deberta-v2-large-japanese-char-wwm', vocab_size=22012, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
# Hugging Faceから試しにデフォルトモデルをダウンロードしてみて、それを音声合成に使ってみる
# model_assetsディレクトリにダウンロードされます

from pathlib import Path
from huggingface_hub import hf_hub_download


model_file = "jvnv-F1-jp/jvnv-F1-jp_e160_s14000.safetensors"
config_file = "jvnv-F1-jp/config.json"
style_file = "jvnv-F1-jp/style_vectors.npy"

for file in [model_file, config_file, style_file]:
    print(file)
    hf_hub_download("litagin/style_bert_vits2_jvnv", file, local_dir="model_assets")

jvnv-F1-jp/jvnv-F1-jp_e160_s14000.safetensors
jvnv-F1-jp/config.json
jvnv-F1-jp/style_vectors.npy


In [7]:
# 上でダウンロードしたモデルファイルを指定して音声合成のテスト

from style_bert_vits2.tts_model import TTSModel

assets_root = Path("model_assets")

model = TTSModel(
    model_path=assets_root / model_file,
    config_path=assets_root / config_file,
    style_vec_path=assets_root / style_file,
    device="cpu",
)

In [8]:
from IPython.display import Audio, display

sr, audio = model.infer(text="私の名前はカワカミたつやです。よろしくお願いします！")
display(Audio(audio, rate=sr))

11-19 00:37:26 |  INFO  | tts_model.py:259 | Start generating audio data from text:
私の名前はカワカミたつやです。よろしくお願いします！
11-19 00:37:26 |  INFO  | infer.py:24 | Using JP-Extra model


/Users/Aporo/Study/python/text-to-speach/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


11-19 00:37:28 |  INFO  | safetensors.py:50 | Loaded 'model_assets/jvnv-F1-jp/jvnv-F1-jp_e160_s14000.safetensors' (iteration 160)
11-19 00:37:32 |  INFO  | tts_model.py:324 | Audio data generated successfully


In [9]:
# 上でダウンロードしたモデルファイルを指定して音声合成のテスト

from style_bert_vits2.tts_model import TTSModel

assets_root = Path("model_assets")
model_file2 = 'jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors'
config_file2 = 'jvnv-F2-jp/config.json'
style_file2 = 'jvnv-F2-jp/style_vectors.npy'

model = TTSModel(
    model_path=assets_root / model_file2,
    config_path=assets_root / config_file2,
    style_vec_path=assets_root / style_file2,
    device="cpu",
)

In [10]:
from IPython.display import Audio, display

sr, audio = model.infer(text="私の名前はカワカミたつやです。よろしくお願いします！")
display(Audio(audio, rate=sr))

11-19 00:37:34 |  INFO  | tts_model.py:259 | Start generating audio data from text:
私の名前はカワカミたつやです。よろしくお願いします！
11-19 00:37:34 |  INFO  | infer.py:24 | Using JP-Extra model
11-19 00:37:35 |  INFO  | safetensors.py:50 | Loaded 'model_assets/jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors' (iteration 166)
11-19 00:37:37 |  INFO  | tts_model.py:324 | Audio data generated successfully


In [11]:
# 上でダウンロードしたモデルファイルを指定して音声合成のテスト

from style_bert_vits2.tts_model import TTSModel

assets_root = Path("model_assets")
model_file3 = 'jvnv-M1-jp/jvnv-M1-jp_e158_s14000.safetensors'
config_file3 = 'jvnv-M1-jp/config.json'
style_file3 = 'jvnv-M1-jp/style_vectors.npy'

model = TTSModel(
    model_path=assets_root / model_file3,
    config_path=assets_root / config_file3,
    style_vec_path=assets_root / style_file3,
    device="cpu",
)

In [12]:
from IPython.display import Audio, display

sr, audio = model.infer(text="私の名前はカワカミたつやです。よろしくお願いします！")
display(Audio(audio, rate=sr))

11-19 00:37:39 |  INFO  | tts_model.py:259 | Start generating audio data from text:
私の名前はカワカミたつやです。よろしくお願いします！
11-19 00:37:39 |  INFO  | infer.py:24 | Using JP-Extra model
11-19 00:37:40 |  INFO  | safetensors.py:50 | Loaded 'model_assets/jvnv-M1-jp/jvnv-M1-jp_e158_s14000.safetensors' (iteration 158)
11-19 00:37:42 |  INFO  | tts_model.py:324 | Audio data generated successfully


In [25]:
# 上でダウンロードしたモデルファイルを指定して音声合成のテスト

from style_bert_vits2.tts_model import TTSModel

assets_root = Path("model_assets")
model_file3 = 'create1/anime-voice-sample_e51_s1000.safetensors'
config_file3 = 'create1/config.json'
style_file3 = 'create1/style_vectors.npy'

model = TTSModel(
    model_path=assets_root / model_file3,
    config_path=assets_root / config_file3,
    style_vec_path=assets_root / style_file3,
    device="cpu",
)

In [27]:
from IPython.display import Audio, display
text = """
これはハギングフェイスからダウンロードしたアニメ声の音声を利用して学習したモデルです。いかがですか？
"""
sr, audio = model.infer(text=text)
display(Audio(audio, rate=sr))

11-19 17:17:55 |  INFO  | tts_model.py:259 | Start generating audio data from text:

これはハギングフェイスからダウンロードしたアニメ声の音声を利用して学習したモデルです。いかがですか？

11-19 17:17:58 |  INFO  | tts_model.py:324 | Audio data generated successfully


In [22]:
# 上でダウンロードしたモデルファイルを指定して音声合成のテスト

from style_bert_vits2.tts_model import TTSModel

assets_root = Path("model_assets")
model_file3 = 'zundamon/zunda-voice_e77_s1000.safetensors'
config_file3 = 'zundamon/config.json'
style_file3 = 'zundamon/style_vectors.npy'

model = TTSModel(
    model_path=assets_root / model_file3,
    config_path=assets_root / config_file3,
    style_vec_path=assets_root / style_file3,
    device="cpu",
)

In [24]:
from IPython.display import Audio, display
text = """
この音声はずんだもんの音声を使って学習されたものです。そっくりですか？
"""
sr, audio = model.infer(text=text)
display(Audio(audio, rate=sr))

11-19 17:16:38 |  INFO  | tts_model.py:259 | Start generating audio data from text:

この音声はずんだもんの音声を使って学習されたものです。そっくりですか？

11-19 17:16:41 |  INFO  | tts_model.py:324 | Audio data generated successfully


In [10]:
# エラー対処: numpy と pyopenjtalk の互換性問題を解決
# numpy.dtype size changed エラーが出る場合は、これらのパッケージを再インストールする

!pip install --upgrade --force-reinstall numpy
!pip install --upgrade --force-reinstall pyopenjtalk

  Using cached numpy-2.3.4-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.3.4-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
Using cached numpy-2.3.4-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.4
    Uninstalling numpy-2.3.4:
      Successfully uninstalled numpy-2.3.4
    Uninstalling numpy-2.3.4:
      Successfully uninstalled numpy-2.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
canary-tts 0.2.1 requires transformers==4.44.2, but you have transformers 4.46.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
canary-tts 0.2.1 requires transforme

## エラーが発生した場合の対処法

上記のセルでエラーが出た場合は、上の対処セルを実行してから **カーネルを再起動** してください。

**手順:**
1. 上のセル（numpy と pyopenjtalk の再インストール）を実行 ✅ **完了**
2. **今すぐカーネルを再起動**（メニューから Kernel → Restart Kernel）
3. 必要なセルを最初から再実行

**注意:** transformersのバージョン警告が出ていますが、ほとんどの場合は動作します。問題があれば後で対処しましょう。